In [1]:
import os
!pip install neo4j
from neo4j import GraphDatabase

senha = "FTIYkBWfEqmr2s-u7m_T6dg3Nm2mfNK9IUTUwVEuvFA"
url = "neo4j+s://0daccb9c.databases.neo4j.io"
usuario = "neo4j"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 4.0 MB/s eta 0:00:00


In [2]:
# Exemplos
# https://gist.github.com/wjgilmore/8ba5f31ef1435dc04c52

class CriarAmizades:
  def __init__(self, uri, user, password):
    self.driver = GraphDatabase.driver(uri, auth=(user, password))

  def close(self):
    self.driver.close()

  def carrega_base(self):
    with self.driver.session() as session:
      session.write_transaction(self._zera_base)
      session.write_transaction(self._cria_amizade)

  @staticmethod
  def _zera_base(tx):
    tx.run("""
MATCH (n)
DETACH DELETE n
""")

  @staticmethod
  def _cria_amizade(tx):
    tx.run("""
CREATE (denise:PESSOA {nome: 'Denise'})
CREATE (kelly:PESSOA {nome: 'Kelly'})
CREATE (joao:PESSOA {nome: 'João'})
CREATE (maria:PESSOA {nome: 'Maria'})
CREATE (david:PESSOA {nome: 'David'})
CREATE (theo:PESSOA {nome: 'Théo'})
CREATE (horacio:PESSOA {nome: 'Horácio'})
CREATE (jose:PESSOA {nome: 'José'})
CREATE (ana:PESSOA {nome: 'Ana'})
CREATE (luiza:PESSOA {nome: 'Luiza'})
CREATE (roberto:PESSOA {nome: 'Roberto'})
CREATE (felipe:PESSOA {nome: 'Felipe'})
CREATE (carlos:PESSOA {nome: 'Carlos'})
CREATE (julia:PESSOA {nome: 'Júlia'})
CREATE (amanda:PESSOA {nome: 'Amanda'})
CREATE (ricardo:PESSOA {nome: 'Ricardo'})
CREATE (gabriel:PESSOA {nome: 'Gabriel'})


CREATE (charlie:CACHORRO {nome: 'Charlie', raca: "labrador"})
CREATE (bobby:CACHORRO {nome: 'Bobby', raca: "corgi"})
CREATE (mia:CACHORRO {nome: 'Mia', raca: "srd"})
CREATE (valentina:CACHORRO {nome: 'Valentina', raca: "corgi"})
CREATE (lola:CACHORRO {nome: 'Lola', raca: "poodle"})
CREATE (rex:CACHORRO {nome: 'Rex', raca: "pastor-alemão"})
CREATE (lady:CACHORRO {nome: 'Lady', raca: "golden retriever"})
CREATE (frodo:CACHORRO {nome: 'Frodo', raca: "beagle"})
CREATE (bella:CACHORRO {nome: 'Bella', raca: "bulldog"})

CREATE (joao)-[:AMIGO {nivel: "bff"}]->(denise)
CREATE (joao)-[:AMIGO {nivel: "conhecido"}]->(maria)
CREATE (kelly)-[:AMIGO]->(joao)
CREATE (kelly)-[:AMIGO]->(denise)
CREATE (maria)-[:AMIGO {nivel: "bff"}]->(denise)
CREATE (david)-[:AMIGO]->(kelly)
CREATE (david)-[:AMIGO]->(joao)
CREATE (david)-[:AMIGO]->(theo)
CREATE (david)-[:AMIGO]->(jose)
CREATE (david)-[:AMIGO]->(horacio)
CREATE (jose)-[:AMIGO]->(horacio)
CREATE (carlos)-[:AMIGO {nivel: "conhecido"}]->(julia)
CREATE (carlos)-[:AMIGO {nivel: "bff"}]->(ana)
CREATE (amanda)-[:AMIGO]->(julia)
CREATE (amanda)-[:AMIGO]->(ricardo)
CREATE (gabriel)-[:AMIGO {nivel: "bff"}]->(ricardo)
CREATE (gabriel)-[:AMIGO]->(felipe)
CREATE (julia)-[:AMIGO]->(luiza)
CREATE (ana)-[:AMIGO {nivel: "conhecido"}]->(luiza)
CREATE (ana)-[:AMIGO {nivel: "bff"}]->(maria)
CREATE (roberto)-[:AMIGO]->(felipe)
CREATE (luiza)-[:AMIGO]->(roberto)
CREATE (felipe)-[:AMIGO {nivel: "conhecido"}]->(luiza)
CREATE (ana)-[:AMIGO {nivel: "bff"}]->(kelly)

CREATE (maria)-[:DONO]->(mia)
CREATE (maria)-[:DONO]->(bobby)
CREATE (denise)-[:DONO]->(charlie)
CREATE (kelly)-[:DONO]->(valentina)
CREATE (carlos)-[:DONO]->(lady)
CREATE (amanda)-[:DONO]->(lola)
CREATE (gabriel)-[:DONO]->(rex)
CREATE (ana)-[:DONO]->(frodo)
CREATE (luiza)-[:DONO]->(bella)
""")


db = CriarAmizades(url, usuario, senha)
db.carrega_base()
db.close()
print("base carregada")

<ipython-input-2-b05cf15b43ec>:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._zera_base)
<ipython-input-2-b05cf15b43ec>:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._cria_amizade)


base carregada


# Consultas

In [3]:
class Neo4jConnection:
  def __init__(self, uri, user, pwd):
    self.__uri = uri
    self.__user = user
    self.__pwd = pwd
    self.__driver = None
    try:
        self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
    except Exception as e:
        print("Failed to create the driver:", e)

  def close(self):
    if self.__driver is not None:
        self.__driver.close()

  def query(self, query, parameters=None, db=None):
    assert self.__driver is not None, "Driver not initialized!"
    session = None
    response = None
    try:
      session = self.__driver.session(database=db) if db is not None else self.__driver.session()
      response = list(session.run(query, parameters))
    except Exception as e:
      print("Query failed:", e)
    finally:
      if session is not None:
        session.close()
    return response

conn = Neo4jConnection(uri=url,
                       user=usuario,
                       pwd=senha)

def consultar(query):
  return conn.query(query)
  # for line in conn.query(query):
  #   print(dict(line))

In [5]:
#####
# Match (a:Nó)-[r:Relacao]->(b:Nó)
####

print("Amigos de João:\n")

consultar("""
MATCH (joao:PESSOA {nome: 'João'})-[tipo:AMIGO]->(amigo:PESSOA)
RETURN joao.nome, amigo.nome, tipo.nivel
""")

Amigos de João:



[<Record joao.nome='João' amigo.nome='Denise' tipo.nivel='bff'>,
 <Record joao.nome='João' amigo.nome='Maria' tipo.nivel='conhecido'>]

In [4]:
print("\nSó os melhores amigos de João:\n")

consultar("""
MATCH (joao:PESSOA {nome: 'João'})-[tipo:AMIGO {nivel: "bff"}]->(amigo:PESSOA)
RETURN joao.nome, amigo.nome AS amigo_nome, tipo.nivel
""")


Só os melhores amigos de João:



[<Record joao.nome='João' amigo_nome='Denise' tipo.nivel='bff'>]

In [6]:
print("\nAmigos de amigos de João:\n")

consultar("""
MATCH (joao:PESSOA {nome: 'João'})-[:AMIGO]->()-[:AMIGO]->(amigo_de_amigo:PESSOA)
RETURN joao.nome, amigo_de_amigo.nome
""")


Amigos de amigos de João:



[<Record joao.nome='João' amigo_de_amigo.nome='Denise'>]

In [7]:
print("\nQuem gosta de Denise:\n")

consultar("""
MATCH (pessoa:PESSOA)-[:AMIGO]->(denise:PESSOA {nome: 'Denise'})
RETURN pessoa.nome
""")


Quem gosta de Denise:



[<Record pessoa.nome='Kelly'>,
 <Record pessoa.nome='João'>,
 <Record pessoa.nome='Maria'>]

In [8]:
print("\nAmigos que começam por J:\n")

consultar("""
MATCH (pessoa:PESSOA)-[:AMIGO]->(amigo)
WHERE pessoa.nome IN ['Théo', 'Kelly'] AND amigo.nome =~ 'J.*'
RETURN pessoa.nome, amigo.nome
""")


Amigos que começam por J:



[<Record pessoa.nome='Kelly' amigo.nome='João'>]

In [ ]:
print("\nQuem gosta de Denise:\n")

consultar("""
MATCH (pessoa:PESSOA)-[:AMIGO]->(denise: PESSOA{nome: 'Denise'})
RETURN pessoa.nome
""")


Quem gosta de Denise:



[<Record pessoa.nome='Kelly'>,
 <Record pessoa.nome='João'>,
 <Record pessoa.nome='Maria'>]

In [ ]:
# https://neo4j.com/docs/cypher-manual/current/syntax/patterns/#cypher-pattern-label

print("\nQuantas pessoas têm 2 gostam de alguém:\n")

consultar("""
MATCH (pessoa:PESSOA)-[*2]->()
RETURN DISTINCT pessoa.nome
""")


Quantas pessoas têm 2 gostam de alguém:



[<Record pessoa.nome='Kelly'>,
 <Record pessoa.nome='João'>,
 <Record pessoa.nome='Maria'>,
 <Record pessoa.nome='David'>,
 <Record pessoa.nome='Ana'>,
 <Record pessoa.nome='Luiza'>,
 <Record pessoa.nome='Roberto'>,
 <Record pessoa.nome='Felipe'>,
 <Record pessoa.nome='Carlos'>,
 <Record pessoa.nome='Júlia'>,
 <Record pessoa.nome='Amanda'>,
 <Record pessoa.nome='Gabriel'>]

In [ ]:
print("\nPessoas com 2 amigos:\n")

consultar("""
MATCH (pessoa:PESSOA)-[:AMIGO]-(amigo:PESSOA)
WITH pessoa, count(amigo) AS qtdAmigos
WHERE qtdAmigos = 2
RETURN pessoa.nome, qtdAmigos
""")


Pessoas com 2 amigos:



[<Record pessoa.nome='Horácio' qtdAmigos=2>,
 <Record pessoa.nome='José' qtdAmigos=2>,
 <Record pessoa.nome='Roberto' qtdAmigos=2>,
 <Record pessoa.nome='Carlos' qtdAmigos=2>,
 <Record pessoa.nome='Amanda' qtdAmigos=2>,
 <Record pessoa.nome='Ricardo' qtdAmigos=2>,
 <Record pessoa.nome='Gabriel' qtdAmigos=2>]

In [ ]:
print("\nQuantas pessoas têm 2 a 3 gostam de alguém:\n")

consultar("""
MATCH (pessoa:PESSOA)-[*2..3]->()
RETURN DISTINCT pessoa.nome
""")


Quantas pessoas têm 2 a 3 gostam de alguém:



[<Record pessoa.nome='Kelly'>,
 <Record pessoa.nome='João'>,
 <Record pessoa.nome='Maria'>,
 <Record pessoa.nome='David'>,
 <Record pessoa.nome='Ana'>,
 <Record pessoa.nome='Luiza'>,
 <Record pessoa.nome='Roberto'>,
 <Record pessoa.nome='Felipe'>,
 <Record pessoa.nome='Carlos'>,
 <Record pessoa.nome='Júlia'>,
 <Record pessoa.nome='Amanda'>,
 <Record pessoa.nome='Gabriel'>]